In [1]:
import pandas as pd

In [2]:
def open_csv(file_path):
    """
    Opens and reads a CSV file using pandas.

    Parameters:
    file_path (str): The path to the CSV file.

    Returns:
    DataFrame: A pandas DataFrame containing the CSV data.
    """
    try:
        df = pd.read_csv(file_path)
        return df
    except FileNotFoundError:
        print(f"The file at {file_path} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [3]:
#File paths for each CSV
apple_store_path = (r"../data/Apple store/AppleStore.csv")
googleplay_store_path = (r"../data/Google Playstore/googleplaystore.csv")

In [4]:
#FOR LATER LOL
#apple_desc_path = "data\Apple store\appleStore_description.csv"
#googleplay_reviews_path = "data\Google Playstore\googleplaystore_user_reviews.csv"
#apple_desc_df = open_csv_with_pandas(apple_desc_path)
#google_reviews_df = open_csv_with_pandas(googleplay_reviews_path)

In [5]:
#DF for each CSV
apple_store_df = open_csv(apple_store_path)
google_store_df = open_csv(googleplay_store_path)

In [6]:
def explore_dataframe(df):
    """
    Explores a pandas DataFrame by checking for missing values, 
    printing its data types, dimensions, and the first few rows.

    Parameters:
    df (pd.DataFrame): The DataFrame to explore.
    Returns:
    None
    """
    if df.isnull().values.any():
        null_summary = df.isnull().sum()
        print("The Dataframe has Missing Values.")
        print("Number of missing values in each column:")
        print(null_summary[null_summary > 0])
    else:
        print("There are no missing values in the DataFrame.")
    print("\nDimensions (rows, columns):")
    print(df.shape)
    print("Data Types (dtypes):")
    print(df.dtypes)
    print(f"\nFirst 5 rows of the DataFrame:")
    print(df.head().to_string(index=False))

In [7]:
explore_dataframe(apple_store_df)

There are no missing values in the DataFrame.

Dimensions (rows, columns):
(7197, 17)
Data Types (dtypes):
Unnamed: 0            int64
id                    int64
track_name           object
size_bytes            int64
currency             object
price               float64
rating_count_tot      int64
rating_count_ver      int64
user_rating         float64
user_rating_ver     float64
ver                  object
cont_rating          object
prime_genre          object
sup_devices.num       int64
ipadSc_urls.num       int64
lang.num              int64
vpp_lic               int64
dtype: object

First 5 rows of the DataFrame:
 Unnamed: 0        id                                         track_name  size_bytes currency  price  rating_count_tot  rating_count_ver  user_rating  user_rating_ver    ver cont_rating  prime_genre  sup_devices.num  ipadSc_urls.num  lang.num  vpp_lic
          1 281656475                                    PAC-MAN Premium   100788224      USD   3.99             21292 

In [8]:
explore_dataframe(google_store_df)

The Dataframe has Missing Values.
Number of missing values in each column:
Rating            1474
Type                 1
Content Rating       1
Current Ver          8
Android Ver          3
dtype: int64

Dimensions (rows, columns):
(10841, 13)
Data Types (dtypes):
App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

First 5 rows of the DataFrame:
                                               App       Category  Rating Reviews Size    Installs Type Price Content Rating                    Genres     Last Updated        Current Ver  Android Ver
    Photo Editor & Candy Camera & Grid & ScrapBook ART_AND_DESIGN     4.1     159  19M     10,000+ Free     0       Everyone              Art & Design 

### Since we are only interested in the free apps, lets filter apps where, in the apple store, the price is 0.00, and in the google store, the type is free.

In [9]:
apple_df = apple_store_df[apple_store_df['price']== 0.00]

In [10]:
google_df = google_store_df[google_store_df['Type'] == 'Free']

In [11]:
apple_df.shape

(4056, 17)

In [12]:
google_df.shape

(10039, 13)

In [13]:
# Liberating some memory by deleting the dataframes with the raw data.
del apple_store_df
del google_store_df

### We are not interested in non-ascii character apps, so we will remove those as well 

In [14]:
def remove_non_ascii_rows(df, column):
    """
    Removes rows from the DataFrame that contain non-ASCII characters in the specified column.

    Parameters:
    df (pd.DataFrame): The DataFrame to filter.
    column (str): The name of the column to check for non-ASCII characters.

    Returns:
    pd.DataFrame: A DataFrame with rows containing non-ASCII characters removed.
    """
    # Create a mask for rows where the specified column contains only ASCII characters
    ascii_mask = df[column].apply(lambda x: all(ord(c) < 128 for c in str(x)))

    # Filter the DataFrame using the mask
    filtered_df = df[ascii_mask]

    print(f"Number of rows after removing non-ASCII characters: {filtered_df.shape[0]}")
    return filtered_df

In [15]:
apple_df = remove_non_ascii_rows(apple_df, 'track_name')

Number of rows after removing non-ASCII characters: 2922


In [16]:
google_df = remove_non_ascii_rows(google_df, 'App')

Number of rows after removing non-ASCII characters: 9488


### Check duplicates.

In [17]:
def find_and_sort_exact_duplicates(df):
    """
    Finds and displays all exact duplicated rows, sorted by the first column.

    Parameters:
    df (pd.DataFrame): The DataFrame to check for exact duplicates.

    Returns:
    pd.DataFrame: A DataFrame containing all exact duplicated rows.
    """
    # Find exact duplicated rows (identical across all columns)
    duplicated_mask = df.duplicated(keep=False)  # keep=False shows all duplicates
    
    # Filter DataFrame to show only exact duplicated rows
    exact_duplicated_rows = df[duplicated_mask]
    
    if not exact_duplicated_rows.empty:
        # Sort the duplicated rows by the first column (or any column of choice)
        exact_duplicated_rows_sorted = exact_duplicated_rows.sort_values(by=exact_duplicated_rows.columns[0])
        
        print(f"Number of exact duplicated rows: {exact_duplicated_rows_sorted.shape[0]}")
        # Use to_string to print the DataFrame without index for clean formatting
        print(exact_duplicated_rows_sorted.to_string(index=False))
        
        return exact_duplicated_rows_sorted
    else:
        print("No exact duplicates found.")
        return None

In [18]:
apple_duplicates_sorted = find_and_sort_exact_duplicates(apple_df)

if apple_duplicates_sorted is not None:
    print(apple_duplicates_sorted)

No exact duplicates found.


In [19]:
google_duplicates_sorted = find_and_sort_exact_duplicates(google_df)

if google_duplicates_sorted is not None:
    print(google_duplicates_sorted)

Number of exact duplicated rows: 788
                                               App            Category  Rating  Reviews               Size       Installs Type Price Content Rating                          Genres       Last Updated              Current Ver        Android Ver
                             10 Best Foods for You  HEALTH_AND_FITNESS     4.0     2490               3.8M       500,000+ Free     0   Everyone 10+                Health & Fitness  February 17, 2017                      1.9       2.3.3 and up
                             10 Best Foods for You  HEALTH_AND_FITNESS     4.0     2490               3.8M       500,000+ Free     0   Everyone 10+                Health & Fitness  February 17, 2017                      1.9       2.3.3 and up
                        1800 Contacts - Lens Store             MEDICAL     4.7    23160                26M     1,000,000+ Free     0       Everyone                         Medical      July 27, 2018                    7.4.1         5.

In [22]:
def remove_duplicates_keep_max_reviews(df):
    """
    Removes duplicates from the DataFrame where the 'App' name is the same, 
    keeping the row with the highest number of 'Reviews'.

    Parameters:
    df (pd.DataFrame): The DataFrame to remove duplicates from.

    Returns:
    pd.DataFrame: A DataFrame with duplicates removed, keeping the highest 'Reviews' for each 'App'.
    """
    # Convert 'Reviews' column to numeric (if necessary), forcing invalid parsing to NaN
    df['Reviews'] = pd.to_numeric(df['Reviews'], errors='coerce')
    
    # For each 'App', keep the row with the maximum 'Reviews'
    idx = df.groupby('App')['Reviews'].idxmax()
    
    # Use these indices to filter the DataFrame
    df_filtered = df.loc[idx].reset_index(drop=True)
    
    return df_filtered

In [23]:
google_df_nodupes = remove_duplicates_keep_max_reviews(google_df)

In [24]:
google_df = google_df_nodupes

In [25]:
google_duplicates_sorted = find_and_sort_exact_duplicates(google_df)

if google_duplicates_sorted is not None:
    print(google_duplicates_sorted)

No exact duplicates found.


In [28]:
google_df.shape

(8407, 13)

### Save the clean dfs to csv

In [ ]:
google_df.to_csv(r"D:\HENRY\PAPAS GPM\data\Google Playstore\googleplaystore_cleaned.csv", index=False)


In [30]:
apple_df.to_csv(r"D:\HENRY\PAPAS GPM\data\Apple store\AppleStore_cleaned.csv", index=False)